Test

In [10]:
import numpy as np

assert np.array([1,2,3])@np.array([1,2,3]) == 14, 'Math is not mathing'

In [11]:
print("hallo wereld")

hallo wereld


In [12]:
from mesa import Agent
from mesa import Model

In [134]:
#https://mesa.readthedocs.io/stable/tutorials/intro_tutorial.html

class FED_Agent(Agent):
    """Agent that models the FED (Central bank of US)"""

    def __init__(self, model):
        super().__init__(model)
        self.interest_rate = 4.75 #current

    #test print uit tutorial
    def print_func(self):
        print(f"The interest rate = {str(self.interest_rate)}. ID = {str(self.unique_id)}.")

class Bank_Agent(Agent):
    """Agent that models the banks"""
    def __init__(self, model):
        super().__init__(model)
        self.money_in_banks = 100 #in trillions oid.
        self.profit_precentage = 0 

class Energy_Sector_Agent(Agent):
    """Agent that models the energy sector"""
    def __init__(self, model):
        super().__init__(model)
        self.change_in_energy_price = 0

class Real_Estat_Agent(Agent):
    def __init__(self, model, avarege_cost_house = 250000):
        super().__init__(model)
        self.avarege_cost_house = avarege_cost_house
        
class Model1(Model):
    """Model for attempt 1."""

    def __init__(self, n, seed=None):
        super().__init__(seed=seed)
        self.num_agents = n

        FED_Agent.create_agents(model = self, n = 1)
        Bank_Agent.create_agents(model = self, n = 1)
        Energy_Sector_Agent.create_agents(model = self, n = 1)
        Real_Estat_Agent.create_agents(model = self, n = 1)

    def step(self):
        """Advance the model by one step."""

        #printen van parameters
        for a in self.agents:
            keys = list(a.__dict__.keys())
            keys = [keys[i] for i in range(len(keys)) if i >=3]
            for key in keys:
                print(key, '=', a.__dict__[key])

In [135]:
#running model
starter_model = Model1(10)
# 1 step
for _ in range(1):
    starter_model.step()

interest_rate = 4.75
money_in_banks = 100
profit_precentage = 0
change_in_energy_price = 0
avarege_cost_house = 250000
